# House Data Lookup using Zillow API

> In this project I am building upon the authors work. This will give estimated profit for a potential buyer to purchase the home as a rental property

Click [HERE](https://www.youtube.com/watch?v=o8Dfa3hUHtc) and [HERE](https://github.com/analyticsariel/projects/blob/master/property_data/zillow_property_data_api.ipynb) to see the references and templates I used for this work.

### Import Packages

In [1]:
from googlesearch import search
import pandas as pd
import requests
import json
import time
import io
import plotly.express as px

In [2]:
# Set up Notebook

# show all columns
pd.set_option('display.max_columns', None)

## Run the Google Search for the URL of a property

In [17]:
# Create variable for search criteria / Address
# 7023 Calamar Drive Fayetteville NC 28314
property_address = "6520 Roaring Creek, Argyle TX 76226"
query = ' zillow homedetails' + property_address

In [18]:
# Create a list of the top 3 search results

results = search(query, num_results=3 )
results_list = [u for u in results] 
results_list


ConnectTimeout: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=%2Bzillow%2Bhomedetails6520%2BRoaring%2BCreek%2C%2BArgyle%2BTX%2B76226&num=5&hl=en&start=0 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028AAB89B410>, 'Connection to www.google.com timed out. (connect timeout=5)'))

In [5]:
# Since "Zillow Home Details" is in the query it should be the number one result
url = results_list[0]
print(url)

https://www.zillow.com/homedetails/6512-Roaring-Crk-Argyle-TX-76226/243581467_zpid/


> The ZPID is what the Zillow API uses to find the data. The ZPID is the numeric part of the url ending with _zpid/

In [6]:
# Get the ZPID to search the property using the API
zpid = [x for x in url.split('/') if 'zpid' in x][0].split('_')[0]
print('Zpid of the property is:', zpid)


Zpid of the property is: 243581467


# Use API to get property details

In [7]:
# This code comes directly from rapidapi.com

url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"zpid":zpid}

headers = {
"X-RapidAPI-Key": "8246fc7aefmshce6cca477ba63adp1661d4jsn1f03c003e6b2",
"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

response.status_code
# 200 indicates successful request

200

### View Property Information

In [8]:
# Here is all of the data from the address listed

response.json()

{'listingProvider': {'enhancedVideoURL': None,
  'showNoContactInfoMessage': False,
  'postingGroupName': None,
  'isZRMSourceText': None,
  'showLogos': None,
  'logos': {},
  'sourceText': None,
  'title': 'Listing Provided by',
  'disclaimerText': None,
  'postingWebsiteURL': None,
  'agentLicenseNumber': None,
  'postingWebsiteLinkText': 'See listing website',
  'enhancedDescriptionText': None,
  'agentName': None},
 'zpid': 243581467,
 'buildingPermits': None,
 'propertyTaxRate': 1.77,
 'contact_recipients': [{'agent_reason': 1,
   'zpro': None,
   'recent_sales': 2,
   'review_count': 270,
   'display_name': 'Swapnil Sharma',
   'zuid': 'X1-ZUysm064vqjqx5_17jmc',
   'rating_average': 4.9,
   'badge_type': 'Premier Agent',
   'phone': {'prefix': '225', 'areacode': '214', 'number': '3529'},
   'image_url': 'https://photos.zillowstatic.com/h_n/IS72xokfxyoimw0000000000.jpg'}],
 'zipcode': '76226',
 'openHouseSchedule': {},
 'longitude': -97.199196,
 'zestimateLowPercent': '5',
 'addr

### Create Pandas DF 

In [9]:
# transform data to pandas dataframe
df_property_detail = pd.json_normalize(data=response.json())
print('Num of rows:', len(df_property_detail))
print('Num of cols:', len(df_property_detail.columns))
df_property_detail.head()

Num of rows: 1
Num of cols: 291


,zpid,buildingPermits,propertyTaxRate,contact_recipients,zipcode,longitude,zestimateLowPercent,cityId,timeOnZillow,url,zestimate,imgSrc,description,price,livingAreaValue,taxHistory,brokerId,stateId,streetAddress,solarPotential,dateSold,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,rentZestimate,bathrooms,annualHomeownersInsurance,state,propertyTypeDimension,building,yearBuilt,brokerageName,pageViewCount,county,monthlyHoaFee,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,livingArea,favoriteCount,schools,zestimateHighPercent,mlsid,countyFIPS,city,providerListingID,country,currency,isListedByOwner,contingentListingType,listingProvider.enhancedVideoURL,listingProvider.showNoContactInfoMessage,listingProvider.postingGroupName,listingProvider.isZRMSourceText,listingProvider.showLogos,listingProvider.sourceText,listingProvider.title,listingProvider.disclaimerText,listingProvider.postingWebsiteURL,listingProvider.agentLicenseNumber,listingProvider.postingWebsiteLinkText,listingProvider.enhancedDescriptionText,listingProvider.agentName,address.community,address.city,address.state,address.neighborhood,address.subdivision,address.streetAddress,address.zipcode,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.builderModel,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.totalActualRent,resoFacts.offerReviewDate,resoFacts.horseYN,resoFacts.buyerAgencyCompensationType,resoFacts.belowGradeFinishedArea,resoFacts.feesAndDues,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.carportParkingCapacity,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.gas,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.roadSurfaceType,resoFacts.subdivisionName,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.bathroomsOneQuarter,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.bathrooms,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherParking,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.heating,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.hasAttachedGarage,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.fireplaceFeatures,resoFacts.hoaFeeTotal,resoFacts.doorFeatures,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.taxAnnualAmount,resoFacts.listAOR,resoFacts.buildingName,resoFacts.attic,resoFacts.mainLevelBedrooms,resoFacts.elementarySchool,resoFacts.virtualTour,resoFacts.hasCarport,resoFacts.canRaiseHorses,resoFacts.hasLandLease,resoFacts.yearBuiltEffective,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.hasHomeWarranty,resoFacts.entryLocation,resoFacts.yearBuilt,resoFacts.propertySubType,resoFacts.propertyCondition,resoFacts.utilities,resoFacts.parcelNumber,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.isNewConstruction,resoFacts.otherStructures,resoFacts.livingArea,resoFacts.livingAreaRangeUnits,resoFacts.buildingArea,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.isSeniorCommunity,resoFacts.foundationDetails,resoFacts.frontageLength,resoFacts.exteriorFeatures,resoFacts.associationAmenities,resoFacts.buildingFeatures,resoFacts.hasCooling,resoFacts.buildingAreaSource,resoFacts.parkingFeatures,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.bathroomsFull,resoFac

### Pull the most recent tax rate on the property

In [10]:
taxHistory = df_property_detail['taxHistory'].iloc[0]

# Sort the tax history by 'time' in descending order to get the most recent entry
sorted_taxHistory = sorted(taxHistory, key=lambda x: x['time'], reverse=True)

# Get the most recent taxPaid value
most_recent_taxPaid = sorted_taxHistory[0]['taxPaid']

print('Most recent taxPaid: {}'.format(most_recent_taxPaid))


Most recent taxPaid: 6601.82


# External Data for Inputs
> Most Current Mortgage Rate 

In [11]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the website that provides mortgage rates
response = requests.get('https://www.usbank.com/home-loans/mortgage/mortgage-rates.html')

# Assuming you have the HTML content in a variable named 'html_content'
html_content = '<span class="mortgageSingleRate conventional 30 display large">6.750%</span>'

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the element containing the mortgage rate
rate_element = soup.find('span', class_='mortgageSingleRate')

# Extract the rate from the element
rate = rate_element.text.strip()


numeric_rate = float(rate.strip('%'))
print('Mortgage rate (numeric): {:.4f}'.format(numeric_rate))


Mortgage rate (numeric): 6.7500


## Property Details and Information

In [12]:
#               Property Information

# school lot_size = df_property_detail['resoFacts.lotSize'].iloc[0]
high_school_district = df_property_detail['resoFacts.highSchoolDistrict'].iloc[0]
mid_school_district = df_property_detail['resoFacts.middleOrJuniorSchoolDistrict'].iloc[0]
ele_school_district = df_property_detail['resoFacts.elementarySchoolDistrict'].iloc[0]

bedrooms = df_property_detail['bedrooms'].iloc[0]
bathrooms = df_property_detail['bathrooms'].iloc[0]
year_built = df_property_detail['yearBuilt'].iloc[0]
property_type = df_property_detail['homeType'].iloc[0]
living_area = df_property_detail['resoFacts.livingArea'].iloc[0]
lot_dimensions = df_property_detail['resoFacts.lotSizeDimensions'].iloc[0]
zoning = df_property_detail['resoFacts.zoning'].iloc[0]








## Input Numbers

In [13]:
# Estimated Price
zestimate = df_property_detail['zestimate'].iloc[0]

# Estimated Down Payment
down_payment = zestimate * .4  # Can we make this an input

# Principal
principal = zestimate - down_payment

# Interest Rate
interest_rate = numeric_rate/100  # Annual interest rate  NEED to make this an input

# Loan Term
loan_term = 30  # Loan term in years NEED to make this an Input

##  Now the rent zestimate 
rent_zestimate = df_property_detail['rentZestimate'].iloc[0]

In [14]:
zestimate

389300

## Payment Calculations

In [15]:
# Mortgage Payment Calculation

monthly_interest_rate = interest_rate / 12  # Monthly interest rate
num_payments = loan_term * 12  # Total number of payments

##  Calculate the monthly mortgage payment
monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_payments)

## Other Expenses

# Insurance   
annual_insurance = df_property_detail['annualHomeownersInsurance'].iloc[0]
insurance = annual_insurance / 12

# Taxes
monthly_taxes= most_recent_taxPaid / 12

# Prop Management
property_managment = rent_zestimate * .10

# Maintenance
maintencance_reserves = rent_zestimate * .03

# Vacancy 
vacancy_reserves = rent_zestimate * .03

# HOA
monthly_HOA = df_property_detail['monthlyHoaFee'].iloc[0]

## Total Expenses
total_expenses = monthly_payment + insurance + monthly_taxes + property_managment + maintencance_reserves + vacancy_reserves



####       Monthly Income 
monthly_income = rent_zestimate - total_expenses

## Annual Income
annual_income = monthly_income * 12

In [16]:
print('Rental Income Calculation For: ', property_address)
print('------------------------------------------------------------------')


print('Price: ${:,.0f}'.format(zestimate))
print('Down payment: ${:,.0f}'.format(down_payment))
print('Todays average 30 year rate: {:.2%}'.format(interest_rate))
# print('The payment will be: ${:,.0f}'.format(monthly_payment))

print('The estimated rent is: ${:,.0f}'.format(rent_zestimate))
print('The estimated expenses: ${:,.0f}'.format(total_expenses))


print('\tMortgage payment: ${:,.0f}'.format(monthly_payment)) # Mortgage

if insurance == 0:
    print("\tInsurance rate: Not Available")
else:
    print('\tInsurance: ${:,.0f}'.format(insurance)) # Insurance


print('\tMonthly Taxes: ${:,.0f}'.format(monthly_taxes))# Taxes
print('\tProperty Management: ${:,.0f}'.format(property_managment))# Prop Management
print('\tMaintenance: ${:,.0f}'.format(maintencance_reserves))# Maintenance
print('\tVacancy: ${:,.0f}'.format(vacancy_reserves))# Vacancy


# print('\tMonthly HOA: ${:,.0f}'.format(monthly_HOA))# HOA
if monthly_HOA not in [0, 15000]:
    print("\tHOA Fee N/A")
else:
    print('\tMonthly HOA: ${:,.0f}'.format(monthly_HOA)) # 
    

print('The estimated monthly income: ${:,.0f}'.format(monthly_income))

print('Estimated Annual Income: ${:,.0f}'.format(annual_income))



print("\n") 
print("\n") 
print("\n") 
print("\n") 


print('PROPERTY DETAILS FOR: ', property_address)
print('------------------------------------------------------------------')

print('  High School District: {}'.format( high_school_district))
print('  Middle School District: {}'.format( mid_school_district))
print('  Elementary School District: {}'.format( ele_school_district))

print('  Bedrooms: {}'.format( bedrooms))
print('  Bathrooms: {}'.format( bathrooms))
print('  Year Built: {}'.format( year_built))
print('  Living Area: {}'.format( living_area))
# print('  Lot Size: {}'.format( lot_size))
print('  Lot Dimensions: {}'.format( lot_dimensions))
print('  Zoning: {}'.format( zoning))
print('  Property Type: {}'.format( property_type))
print('  Zestimate: ${:,.0f}'.format( zestimate))
print('  Rent Zestimate: ${:,.0f}'.format( rent_zestimate))





Rental Income Calculation For:  6520 Roaring Creek Argyle TX 76226
------------------------------------------------------------------
Price: $389,300
Down payment: $155,720
Todays average 30 year rate: 6.75%
The estimated rent is: $2,375
The estimated expenses: $2,581
	Mortgage payment: $1,515
	Insurance: $136
	Monthly Taxes: $550
	Property Management: $238
	Maintenance: $71
	Vacancy: $71
	HOA Fee N/A
The estimated monthly income: $-206
Estimated Annual Income: $-2,477








PROPERTY DETAILS FOR:  6520 Roaring Creek Argyle TX 76226
------------------------------------------------------------------
  High School District: None
  Middle School District: None
  Elementary School District: None
  Bedrooms: 3
  Bathrooms: 2
  Year Built: 2017
  Living Area: 1,738 sqft
  Lot Dimensions: None
  Zoning: None
  Property Type: SINGLE_FAMILY
  Zestimate: $389,300
  Rent Zestimate: $2,375
